In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import json
import pandas as pd
import sys
import difflib
import fidelity_check

In [4]:
screenshot_similarity = json.load(open("../record_replay/screenshots/onload_screenshot_similarity.json", 'r'))
screenshot_similarity = {s['directory']: s['screenshot_similarity'] for s in screenshot_similarity for s in screenshot_similarity}

In [6]:
def check_fidelity(dirr):
    screenshot_simi = screenshot_similarity[dirr.split('/')[-1]]
    live_html = open(f"{dirr}/live.html").read()
    archive_html = open(f"{dirr}/archive.html").read()
    live_element = json.load(open(f"{dirr}/live_elements.json"))
    archive_element = json.load(open(f"{dirr}/archive_elements.json"))
    my_fidelity = fidelity_check.verify(live_html, live_element, archive_html, archive_element)
    print(screenshot_simi >= 1, my_fidelity)

# check_fidelity('testcases/theftaz.azag.gov_1')

#### Test difflib

In [27]:
e1 = fidelity_check.htmlElement({'xpath': '/a/b', 'text': 'hello'})
e2 = fidelity_check.htmlElement({'xpath': '/a/b', 'text': 'hello'})
e1 == e2

/a/b /a/b True


True

## Check archive differences

In [9]:
dirr = 'testcases/www.slideshare.net_1'
dirr = '../execution_match/examples/eta.lbl.gov_2'

live_element = json.load(open(f"{dirr}/live_elements.json"))
archive_element = json.load(open(f"{dirr}/archive_elements.json"))
# fidelity_check.verify( live_element, archive_element)
live_unique, archive_unique = fidelity_check.diff(live_element, archive_element, returnHTML=False)
print(len(live_unique), len(archive_unique))
for i in live_unique:
    print("live:\n", i)
for i in archive_unique:
    print("archive:\n", i)

live_unique number [2, 1, 1, 1, 1, 1, 1, 1]
archive_unique number [2, 1, 1, 1, 1, 1, 7, 1]
8 8
live:
                         <div class="media media--blazy media--bundle--slideshow-carousel media--slick media--responsive media--image is-b-loaded" style="" tabindex="0">
                            <img class="media__image media__element b-lazy b-responsive img-zoomable b-loaded" src="https://eta.lbl.gov/sites/default/files/styles/slideshow/public/slideshow_images/SP%20Cover_Web.jpg?itok=u6DKd1j9" alt="Berkeley Lab Skyline Graphic" style="cursor: zoom-in;">

live:
                                               <path class="ytp-svg-fill" d="M 16.68,.99 C 13.55,1.03 7.02,1.16 4.99,1.68 c -1.49,.4 -2.59,1.6 -2.99,3 -0.69,2.7 -0.68,8.31 -0.68,8.31 0,0 -0.01,5.61 .68,8.31 .39,1.5 1.59,2.6 2.99,3 2.69,.7 13.40,.68 13.40,.68 0,0 10.70,.01 13.40,-0.68 1.5,-0.4 2.59,-1.6 2.99,-3 .69,-2.7 .68,-8.31 .68,-8.31 0,0 .11,-5.61 -0.68,-8.31 -0.4,-1.5 -1.59,-2.6 -2.99,-3 C 29.11,.98 18.40,.99 18.40,.99 c

In [99]:
# * Generate testcase
diff = {"live": [], "archive": []}
for lu in live_unique:
    first_line = lu.strip().split('\n')[0]
    diff['live'].append({"root": first_line, "numlines": len(lu.strip().split('\n'))})
for au in archive_unique:
    first_line = au.strip().split('\n')[0]
    diff['archive'].append({"root": first_line, "numlines": len(au.strip().split('\n'))})
print(json.dumps(diff, indent=4))

{
    "live": [
        {
            "root": "<div class=\"media media--blazy media--bundle--slideshow-carousel media--slick media--responsive media--image is-b-loaded\" style=\"\" tabindex=\"0\">",
            "numlines": 2
        },
        {
            "root": "<div class=\"ytp-title-channel\">",
            "numlines": 1
        },
        {
            "root": "<div class=\"ytp-title-channel\">",
            "numlines": 1
        }
    ],
    "archive": [
        {
            "root": "<div class=\"media media--blazy media--bundle--slideshow-carousel media--slick media--loading is-b-loading media--responsive media--image\" style=\"\" tabindex=\"0\">",
            "numlines": 2
        },
        {
            "root": "<button class=\"ytp-watch-later-button ytp-button ytp-show-watch-later-title\" data-tooltip-opaque=\"true\" title=\"Watch later\" aria-label=\"Watch later\">",
            "numlines": 7
        }
    ]
}


### Check associated writes from diffs

In [7]:
dirr = '../execution_match/examples/www.slideshare.net'

live_element = json.load(open(f"{dirr}/live_new_elements.json"))
archive_element = json.load(open(f"{dirr}/archive_elements.json"))
# fidelity_check.verify( live_element, archive_element)
live_unique, archive_unique = fidelity_check.diff(live_element, archive_element)
print(len(live_unique), len(archive_unique))

for 

live_unique number [11, 9, 1, 1]
archive_unique number [1, 1]
4 2
